In [0]:
%tensorflow_version 1.x

In [0]:
### Do just once In google Colab left pane: ###
# Upload into Files:
  # neural.zip
  # data.zip
  # helpers_models.py
  # helpers_strings.py

In [0]:
### run just once: ### -- downloads tardan 15 min aprox
# !unzip data.zip
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

In [4]:
!unzip -n glove.6B.zip
!unzip -n glove.twitter.27B.zip
!unzip -o neural.zip

Archive:  glove.6B.zip
Archive:  glove.twitter.27B.zip
Archive:  neural.zip
 extracting: neural/__init__.py      
  inflating: neural/architectures.py  
 extracting: neural/core/__init__.py  
  inflating: neural/core/helpers.py  
  inflating: neural/core/training.py  


In [5]:
import pandas as pd
import numpy as np

import helpers_models as hm

from neural.architectures import build_gru
from neural.core.training import full_nn

Using TensorFlow backend.


In [0]:
RUN_SANITY = False
NEW_RANDOM_SPLIT = True
LOAD_PRETRAINED_VECS = True # False no genera el dict de embeddings (xq tarda unos min)

In [7]:
if NEW_RANDOM_SPLIT:
  semilla = np.random.randint(1,9999)
else:
  semilla = 800
rng = np.random.RandomState(semilla)
print(semilla)

8156


In [0]:
datos_raw = hm.read_tagged_data()
datos = hm.clean_tagged_data(datos_raw)
X = datos['text']
y = datos['target']
X_train, X_val, y_train, y_val = hm.split_data(X, y, 0.2, rng)

In [9]:
if LOAD_PRETRAINED_VECS:
  # load pretrained embeddings glove_twitter
  embeddings_twitter = {}
  with open('glove.twitter.27B.200d.txt') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, 'f', sep=' ')
      embeddings_twitter[word] = coefs
  # load pretrained embeddings glove_wiki
  embeddings_wiki = {}
  with open('glove.6B.300d.txt') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, 'f', sep=' ')
      embeddings_wiki[word] = coefs

ERROR! Session/line number was not unique in database. History logging moved to new session 68


In [0]:
### GRU sanity check ###
if RUN_SANITY:
  param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
  param_nn = dict(
                        optimizer='adam'
                        ,learn_rate=1e-5
                        ,l2_strength=0.0
                        ,decay_strength=0.0
                        ,momentum=None          
                        ,embeddings=embeddings_twitter
                        ,initializer='he_uniform'  
                  )
  param_train = dict(
                        model_id='sanity'
                        ,epochs=20
                        ,batch_size=32
                        ,early_stopping_n=999999
                        ,decay_factor=0.0
                        ,decay_patience_n=999999
                        ,sanity_check_n=20
                        ,verbose=0
                    )
  param_sanity = dict(**param_tokenizer, **param_nn, **param_train)
  mod_check = full_nn(
                build_gru, X_train, y_train, X_val, y_val, **param_sanity
                ) 

In [0]:
# NN parameters
param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
param_nn = dict(
                      optimizer='adam'
                      ,learn_rate=1e-5
                      ,l2_strength=0.01
                      ,decay_strength=0.0
                      ,momentum=None          
                      ,embeddings=embeddings_wiki
                      ,initializer='he_uniform'
                )
param_train = dict(
                      model_id='GloveWiki01'
                      ,epochs=200
                      ,batch_size=16
                      ,early_stopping_n=20
                      ,decay_factor=0.8
                      ,decay_patience_n=10
                      ,sanity_check_n=None
                      ,verbose=2
                  )
param = dict(**param_tokenizer, **param_nn, **param_train)

# Training NN
mod = full_nn(build_gru, X_train, y_train, X_val, y_val, **param)












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 6002 samples, validate on 1501 samples
Epoch 1/200
 - 6s - loss: 37.7300 - acc: 0.6140 - val_loss: 36.3572 - val_acc: 0.6696

Epoch 00001: val_acc improved from -inf to 0.66955, saving model to GloveWiki01_best.hdf5
- val_f1: 0.5811 - val_rec: 0.5350 - val_prec: 0.6359 
Epoch 2/200
 - 4s - loss: 35.0733 - acc: 0.6533 - val_loss: 33.7922 - val_acc: 0.6869

Epoch 00002: val_acc improved from 0.66955 to 0.68688, saving model to GloveWiki01_best.hdf5
- val_f1: 0.6185 - val_rec: 0.5925 - val_prec: 0.6469 
Epoch 3/200
 - 4s - loss: 32.5852 - acc: 0.6979 - val_loss: 31.3832 - val_acc: 0.7235

Epoch 00003: val_acc improved from 0.68688 to 0.72352, saving model to GloveWiki01_best.hdf5
- val_f1: 0.6765 - val_rec: 0.6750 - val_prec: 0.6781 
Epoch 4/200
 - 4s - loss: 30.2395 - acc: 0.7429 - val_loss: 29.1105 - val_acc: 0.7535

Epoch 00004: val_acc improved from 0.72352 to 0.75350, s

In [0]:
# Agregar RandonSearch (o bayesian para los aventureros)